In [1]:
import sys
#sys.path.insert(1, '/Users/swastik/ophthalmology/Project_Quality_Assurance/From_Bascom/Hold_out/') # noqa
sys.path.insert(1,'/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Application/training/Non_Distributed') # noqa
from import_packages.dataset_class import Dataset
from import_packages.train_val_to_ids import inference_val_to_ids
from import_packages.checkpoint import load_checkpoint
import os
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from efficientnet_pytorch import EfficientNet

In [46]:
pip install efficientnet_pytorch

Note: you may need to restart the kernel to use updated packages.


In [2]:
path_dir = '/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/data_models/chocolate_feather/df_val.csv' # noqa


In [3]:
df = pd.read_csv(path_dir).iloc[:, 2:]
df

,img,Excellent,Good,Adequate,Insuff_for_full,Insuff_for_any,labels,Redfree,other
0,20170120223854648.JPG,1,0,0,0,0,0,NaN,NaN
1,299458_2024651.JPG,0,0,0,1,0,2,NaN,NaN
2,301839_2043723.JPG,1,0,0,0,0,0,NaN,NaN
3,294432_1985275.JPG,1,0,0,0,0,0,NaN,NaN
4,272650_1820786.JPG,0,0,0,1,0,2,NaN,NaN
...,...,...,...,...,...,...,...,...,...
758,324765_2224415.JPG,0,0,0,1,0,2,NaN,NaN
759,007-8834-605.JPG,0,0,0,0,1,2,NaN,NaN
760,20170430193513456.JPG,0,1,0,0,0,1,NaN,NaN
761,257895_1711283.JPG,0,1,0,0,0,1,NaN,NaN


In [4]:
source_dir = '/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Data_Staging/Combined_Dataset_224' # noqa


In [50]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
# Torch RNG
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Python RNG
np.random.seed(seed)
random.seed(seed)
# CuDA Determinism
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:

# %%
batch_size = 2
partition, labels = inference_val_to_ids(csv_file=path_dir, stratify_columns='labels') # noqa
test_set = Dataset(partition['test_set'],labels,root_dir=source_dir,test_transform=True) # noqa
test_loader = torch.utils.data.DataLoader(test_set,shuffle=True,pin_memory=True, num_workers =4, batch_size=batch_size) # noqa

In [6]:
model_transfer = EfficientNet.from_pretrained('efficientnet-b2')
n_inputs = model_transfer._fc.in_features
model_transfer._fc = nn.Linear(n_inputs, 3)

Loaded pretrained weights for efficientnet-b2


In [7]:
#weights = torch.tensor([0.0008, 0.0001, 0.0003])
optimizer = torch.optim.SGD(model_transfer.parameters(), lr=3e-4)
criterion_transfer = nn.CrossEntropyLoss()
scheduler = ReduceLROnPlateau(
            optimizer,
            patience=4,
            factor=0.1,
            mode='min',
            verbose=True
        )


In [8]:
if torch.cuda.is_available():
    use_cuda = True
else:
    use_cuda = False

In [9]:
model = load_checkpoint(checkpoint_path='/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Data_models/chocolate_feather/checkpoint_224.pt',model = model_transfer) # noqa


In [56]:
def accuracy(model, loader, criterion, use_cuda=False):
    """Calculate Accuracy."""
    test_accuracy = 0.0
    test_correct = 0.0
    test_total = 0.0
    model.eval()
    for batch_idx, (data, target) in enumerate(loader):
        # print("data shape:", data.shape)
        # print("target shape:", target.shape)
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # update the average validation loss
        output = model(data).squeeze()
        pred = torch.argmax(output, dim=1)
        test_correct += np.sum(np.squeeze(pred.eq(target.view_as(pred)).cpu().numpy())) # noqa
        test_total += data.size(0)
    test_accuracy = 100. * (test_correct/test_total)
    print(test_accuracy)
    return None

In [69]:
for batch_idx, (data, target) in enumerate(test_loader):
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    print(data.shape)
    output = torch.nn.functional.softmax(model(data),dim=1).squeeze()
    print(output)
    pred = torch.argmax(output, dim=1)
    print(pred.shape)
    print("Next tensor")
    break

torch.Size([2, 3, 224, 224])
tensor([[0.0501, 0.4176, 0.5324],
        [0.2734, 0.5931, 0.1335]], grad_fn=<SqueezeBackward0>)
torch.Size([2])
Next tensor


In [65]:
output = output.detach().numpy()

AttributeError: 'numpy.ndarray' object has no attribute 'detach'

0.38250783

In [31]:
a = torch.randn(2, 3)
pred = torch.argmax(a, dim=1)
pred.shape

torch.Size([2])

In [57]:
accuracy(model, test_loader, criterion_transfer, use_cuda)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [43]:
response = open("/Users/swastik/ophthalmology/Project_Quality_Assurance/Final_QA_FDA/Application/Quality_scanner/gradio_UI/label_names.txt","r") # noqa
labels = [line.rstrip('\n') for line in response]
labels

['Excellent', 'Good_Adequate', 'Insufficient_for_Any_All']